## Pipeline qiime2 BLAST des OTUs sur amplicons Illumina

* importation des données DADA2
Séquences illumina avec plusieurs marqueurs par index, ici 16SV4 et ITS2.
On débruite et déréplique tout en bloc avec DADA2 sous R car c'est beaucoup plus rapide que DADA2 sous environnement qiime2. Puis on importe la table des séquences uniques de DADA2 en artefacts qiime2.

**sous R (DADA2):**
write.table(t(seqtab.nochim), "~/sync/mangroves/data_sequencages/guyane/dada2-analysis/seqtab-nochim.txt", sep="\t", row.names=TRUE, col.names=NA, quote=FALSE)

uniquesToFasta(seqtab.nochim, "~/sync/mangroves/data_sequencages/guyane/dada2-analysis/rep-seqs.fna", ids=colnames(seqtab.nochim))

* sous bash, activation environnement conda qiime2

**Le trimming des amorces (15pb à gauche, 15pb à droite) permet d'accroitre d'environ 30% le nombre de reads filtrés, en diminuant le nombre de chimères**

--> sans trimmer les amorces
"input" "filtered" "denoisedF" "denoisedR" "merged" "nonchim"
"Blancs" 24460 6389 5785 5461 3793 2234
"S1D101" 85622 58201 44011 42027 9794 8924
"S1D102" 99085 65076 52672 50128 11480 10160
"S1D203" 64106 46813 37450 36485 10696 9727
"S1D204" 66110 46613 37775 36016 11683 10316
"S1D205" 91809 63124 50219 49882 14095 12630


--> en trimmant les amorces
"input" "filtered" "denoisedF" "denoisedR" "merged" "nonchim"
"Blancs" 24460 6936 6377 6136 4707 4110
"S1D101" 85622 61596 52863 54327 14146 13034
"S1D102" 99085 68605 61354 61952 18091 16729
"S1D203" 64106 49032 43066 43950 14611 13768
"S1D204" 66110 48630 42887 43691 15766 14586
"S1D205" 91809 65849 57067 59604 18494 17002

* il faut au préalable créer les dossiers /qiime2, /OTU-ITS, /OTU-16S et OTU-16S-taxo
* création de l'artefact rep-seqs.qza issu de seqtab.nochim (renommé en rep-seqs.fna sous DADA2/R)

In [ ]:
##########################################################
### 
### script à reprendre avant dépôt
### le jeu de données LB + SED mélange les 16S et ITS dans les mêmes sorties
### il faut donc les séparer et les traiter séparemment
### deplus les chemins ne sont pas les bons

In [6]:
for i in 1 2 3 4 5
do
   echo "Welcome $i times"
done

Welcome 1 times
Welcome 2 times
Welcome 3 times
Welcome 4 times
Welcome 5 times


In [15]:
## Extraction des 18SV9 à partir des fastq
## conda deactivate

cd /Users/tonyrobinet/Desktop/mangroves_guadeloupe_rawdata/genbank_data_guadeloupe_litterbags
for l in $(ls *.fastq.gz)
do
VAR=${l%.*}
echo "${VAR%.*}"
done

BLANCEXT_R1_final
BLANCEXT_R2_final
BLANCPCR_R1_final
BLANCPCR_R2_final
BT11_R1_final
BT11_R2_final
BT12_R1_final
BT12_R2_final
BT13_R1_final
BT13_R2_final
BT14_R1_final
BT14_R2_final
BT15_R1_final
BT15_R2_final
BT21_R1_final
BT21_R2_final
BT22_R1_final
BT22_R2_final
BT23_R1_final
BT23_R2_final
BT24_R1_final
BT24_R2_final
BT25_R1_final
BT25_R2_final
DT11_R1_final
DT11_R2_final
DT12_R1_final
DT12_R2_final
DT13_R1_final
DT13_R2_final
DT14_R1_final
DT14_R2_final
DT15_R1_final
DT15_R2_final
DT21_R1_final
DT21_R2_final
DT22_R1_final
DT22_R2_final
DT23_R1_final
DT23_R2_final
DT24_R1_final
DT24_R2_final
DT25_R1_final
DT25_R2_final
FT0D_R1_final
FT0D_R2_final
FT0I_R1_final
FT0I_R2_final
FT11_R1_final
FT11_R2_final
FT12_R1_final
FT12_R2_final
FT13_R1_final
FT13_R2_final
FT14_R1_final
FT14_R2_final
FT15_R1_final
FT15_R2_final
GT11_R1_final
GT11_R2_final
GT12_R1_final
GT12_R2_final
GT13_R1_final
GT13_R2_final
GT14_R1_final
GT14_R2_final
GT15_R1_final
GT15_R2_final
GT21_R1_final
GT21_R2_final
GT22

In [25]:
seqkit fa2fq -h

retrieve corresponding FASTQ records by a FASTA file

Attention:
  1. We assume the FASTA file comes from the FASTQ file,
     so they share sequence IDs, and sequences in FASTA
     should be subseq of sequences in FASTQ file.

Usage:
  seqkit fa2fq [flags] 

Flags:
  -f, --fasta-file string      FASTA file)
  -h, --help                   help for fa2fq
  -P, --only-positive-strand   only search on positive strand

Global Flags:
      --alphabet-guess-seq-length int   length of sequence prefix of the first FASTA record based on
                                        which seqkit guesses the sequence type (0 for whole seq)
                                        (default 10000)
      --compress-level int              compression level for gzip, zstd, xz and bzip2. type "seqkit -h"
                                        for the range and default value for each format (default -1)
      --id-ncbi                         FASTA head is NCBI-style, e.g. >gi|110645304|ref|NC_002516.2|
    

In [ ]:
cd /Users/tonyrobinet/Desktop/mangroves_guadeloupe_rawdata/genbank_data_guadeloupe_litterbags

for l in $(ls *.fastq.gz)
do
VAR=${l%.*}
echo "${VAR%.*}"
seqkit fq2fa $VAR.gz -o ${VAR%.*}.fasta
cat ${VAR%.*}.fasta | seqkit amplicon -F CCCTGCCHTTTGTACACAC -R CCTTCYGCAGGTTCACCTAC -r 1:-1 > ${VAR%.*}_18SV9.fasta
echo "${VAR%.*} 1"
cat ${VAR%.*}_18SV9.fasta | awk 'NR%4==1' | sed 's/@//' > ids
seqkit grep -f ids $l > ${l}_18SV9.fastq.gz
echo "${VAR%.*} 2"
rm ${VAR%.*}.fasta
echo "${VAR%.*} 3"
done

In [32]:
cat ${VAR%.*}_18SV9.fasta | awk 'NR%4==1' | sed 's/@//'


cat: BLANCEXT_R2_final_18SV9.fasta: No such file or directory


In [1]:
source activate qiime2-amplicon-2023.9
##conda info --envs

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 


: 1

In [9]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024

qiime tools import \
--input-path ./dada2/seqtabnochim_guadeloupe_LB_final_mai24_notrim.fna \
--type 'FeatureData[Sequence]' \
--output-path ./qiime2/rep-seqs_guadeloupe_LB_16S_18S_ITS.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported ./dada2/seqtabnochim_guadeloupe_LB_final_mai24_notrim.fna as DNASequencesDirectoryFormat to ./qiime2/rep-seqs_guadeloupe_LB_16S_18S_ITS.qza
(qiime2-amplicon-2023.9) 


: 1

* création de l'artefact table.qza par biom

In [11]:
# echo -n "#OTU Table" | cat -  > ./biom-table_guyane_all.txt
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024

biom convert -i ./dada2/seqtabnochim_guadeloupe_LB_final_mai24_notrim.txt -o ./qiime2/table_guadeloupe_LB_16S_18S_ITS.biom --table-type="OTU table" --to-hdf5

qiime tools import \
--input-path ./qiime2/table_guadeloupe_LB_16S_18S_ITS.biom \
--type 'FeatureTable[Frequency]' \
--output-path ./qiime2/table_guadeloupe_LB_16S_18S_ITS.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported ./qiime2/table_guadeloupe_LB_16S_18S_ITS.biom as BIOMV210DirFmt to ./qiime2/table_guadeloupe_LB_16S_18S_ITS.qza
(qiime2-amplicon-2023.9) 


: 1

In [ ]:
### dans DADA2 on a trimmé 15bp des deux côtés des reads pour diminuer le taux de chimères
### du coup on effectue Vsearch et on assigne directement

### ou alors on ne trimme rien et on trie par marqueur, avant Vsearch et assignation (mais on perd plus de la moitié des reads)

In [12]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime feature-classifier extract-reads \
  --i-sequences rep-seqs_guadeloupe_LB_16S_18S_ITS.qza \
  --p-f-primer GTGYCAGCMGCCGCGGTAA \
  --p-r-primer CCGYCAATTYMTTTRAGTTT \
  --p-identity 0.7 \
  --p-n-jobs 6 \
  --o-reads ./OTU-16S/rep-seqs_guadeloupe_LB_16S.qza
  
qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S_18S_ITS.qza \
  --m-metadata-file ./OTU-16S/rep-seqs_guadeloupe_LB_16S.qza \
  --o-filtered-table ./OTU-16S/table_guadeloupe_LB_16S.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureData[Sequence] to: ./OTU-16S/rep-seqs_guadeloupe_LB_16S.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureTable[Frequency] to: ./OTU-16S/table_guadeloupe_LB_16S.qza
(qiime2-amplicon-2023.9) 


: 1

In [14]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime feature-classifier extract-reads \
  --i-sequences rep-seqs_guadeloupe_LB_16S_18S_ITS.qza \
  --p-f-primer CCCTGCCHTTTGTACACAC \
  --p-r-primer CCTTCYGCAGGTTCACCTAC \
  --p-identity 0.7 \
  --p-n-jobs 6 \
  --o-reads ./OTU-18S/rep-seqs_guadeloupe_LB_18S.qza
  
qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S_18S_ITS.qza \
  --m-metadata-file ./OTU-18S/rep-seqs_guadeloupe_LB_18S.qza \
  --o-filtered-table ./OTU-18S/table_guadeloupe_LB_18S.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureData[Sequence] to: ./OTU-18S/rep-seqs_guadeloupe_LB_18S.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureTable[Frequency] to: ./OTU-18S/table_guadeloupe_LB_18S.qza
(qiime2-amplicon-2023.9) 


: 1

In [15]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime feature-classifier extract-reads \
  --i-sequences rep-seqs_guadeloupe_LB_16S_18S_ITS.qza \
  --p-f-primer GTGAATCATCGAATCTTTGAA \
  --p-r-primer TCCTCCGCTTATTGATATGC \
  --p-identity 0.9 \
  --p-n-jobs 6 \
  --o-reads ./OTU-ITS/rep-seqs_guadeloupe_LB_ITS.qza
  
qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S_18S_ITS.qza \
  --m-metadata-file ./OTU-ITS/rep-seqs_guadeloupe_LB_ITS.qza \
  --o-filtered-table ./OTU-ITS/table_guadeloupe_LB_ITS.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureData[Sequence] to: ./OTU-ITS/rep-seqs_guadeloupe_LB_ITS.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureTable[Frequency] to: ./OTU-ITS/table_guadeloupe_LB_ITS.qza
(qiime2-amplicon-2023.9) 


: 1

* VSEARCH produit la table des OTUs (<3%), pour chaque marqueur

In [5]:
## pour obtenir le fasta il faut ouvrir le rep-seqs_XXX.qza avec un utilitaire d'archives
## extraire le fasta du dossier data
## et le renommer en rep-seqs_XXX.fasta

cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-16S/

vsearch --cluster_size rep-seqs_guadeloupe_LB_16S.fasta \
    --threads 8 \
    --id 0.97 \
    --sizeout \
    --fasta_width 0 \
    --qmask soft \
    --centroids rep-seqs-97_guadeloupe_LB_16S.fasta \
    --otutabout rep-seqs-97_guadeloupe_LB_16S.txt

qiime tools import \
--input-path rep-seqs-97_guadeloupe_LB_16S.fasta \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs-97_guadeloupe_LB_16S.qza

qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S.qza \
  --m-metadata-file rep-seqs-97_guadeloupe_LB_16S.qza \
  --o-filtered-table table-97_guadeloupe_LB_16S.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
vsearch v2.22.1_macos_x86_64, 16.0GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file rep-seqs_guadeloupe_LB_16S.fasta 100%  
2794338 nt in 7519 seqs, min 166, max 432, avg 372
Sorting by abundance 100%
Counting k-mers 100% 
Clustering 100%  
Sorting clusters 100%
Writing clusters 100% 
Clusters: 3551 Size min 1, max 73, avg 2.1
Singletons: 2561, 34.1% of seqs, 72.1% of clusters
Writing OTU table (classic) 100%  
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported rep-seqs-97_guadeloupe_LB_16S.fasta as DNASequencesDirectoryFormat to rep-seqs-97_guadeloupe_LB_16S.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureTable[Frequency] to: table-97_guadeloupe_LB_16S.qza
(qiime2-amplicon-2023.9) 


: 1

In [6]:
## d'abord extraire le fasta comme indiqué cellulle plus haut
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-18S/

vsearch --cluster_size rep-seqs_guadeloupe_LB_18S.fasta \
    --threads 8 \
    --id 0.97 \
    --sizeout \
    --fasta_width 0 \
    --qmask soft \
    --centroids rep-seqs-97_guadeloupe_LB_18S.fasta \
    --otutabout rep-seqs-97_guadeloupe_LB_18S.txt

qiime tools import \
--input-path rep-seqs-97_guadeloupe_LB_18S.fasta \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs-97_guadeloupe_LB_18S.qza

qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_18S.qza \
  --m-metadata-file rep-seqs-97_guadeloupe_LB_18S.qza \
  --o-filtered-table table-97_guadeloupe_LB_18S.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
vsearch v2.22.1_macos_x86_64, 16.0GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file rep-seqs_guadeloupe_LB_18S.fasta 100%  
358 nt in 2 seqs, min 171, max 187, avg 179
Sorting by abundance 100%
Counting k-mers 100% 
Clustering 100%  
Sorting clusters 100%
Writing clusters 100% 
Clusters: 2 Size min 1, max 1, avg 1.0
Singletons: 2, 100.0% of seqs, 100.0% of clusters
Writing OTU table (classic) 100%  
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported rep-seqs-97_guadeloupe_LB_18S.fasta as DNASequencesDirectoryFormat to rep-seqs-97_guadeloupe_LB_18S.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureTable[Frequency] to: table-97_guadeloupe_LB_18S.qza
(qiime2-amplicon-2023.9) 


: 1

In [7]:
## d'abord extraire le fasta comme indiqué cellulle plus haut
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-ITS/

vsearch --cluster_size rep-seqs_guadeloupe_LB_ITS.fasta \
    --threads 8 \
    --id 0.97 \
    --sizeout \
    --fasta_width 0 \
    --qmask soft \
    --centroids rep-seqs-97_guadeloupe_LB_ITS.fasta \
    --otutabout rep-seqs-97_guadeloupe_LB_ITS.txt

qiime tools import \
--input-path rep-seqs-97_guadeloupe_LB_ITS.fasta \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs-97_guadeloupe_LB_ITS.qza

qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_ITS.qza \
  --m-metadata-file rep-seqs-97_guadeloupe_LB_ITS.qza \
  --o-filtered-table table-97_guadeloupe_LB_ITS.qza

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
vsearch v2.22.1_macos_x86_64, 16.0GB RAM, 8 cores
https://github.com/torognes/vsearch

Reading file rep-seqs_guadeloupe_LB_ITS.fasta 100%  
139539 nt in 516 seqs, min 169, max 414, avg 270
Sorting by abundance 100%
Counting k-mers 100% 
Clustering 100%  
Sorting clusters 100%
Writing clusters 100% 
Clusters: 225 Size min 1, max 26, avg 2.3
Singletons: 116, 22.5% of seqs, 51.6% of clusters
Writing OTU table (classic) 100%  
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported rep-seqs-97_guadeloupe_LB_ITS.fasta as DNASequencesDirectoryFormat to rep-seqs-97_guadeloupe_LB_ITS.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureTable[Frequency] to: table-97_guadeloupe_LB_ITS.qza
(qiime2-amplicon-2023.9) 


: 1

* exporter les tables d'OTUs

In [9]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime tools export \
    --input-path ./OTU-16S/table-97_guadeloupe_LB_16S.qza \
    --output-path ./OTU-16S
    
biom convert \
    -i ./OTU-16S/feature-table.biom \
    -o ./OTU-16S/feature-table-97_guadeloupe_LB_16S.tsv --to-tsv


qiime tools export \
    --input-path ./OTU-18S/table-97_guadeloupe_LB_18S.qza \
    --output-path ./OTU-18S
    
biom convert \
    -i ./OTU-18S/feature-table.biom \
    -o ./OTU-18S/feature-table-97_guadeloupe_LB_18S.tsv --to-tsv

qiime tools export \
    --input-path ./OTU-ITS/table-97_guadeloupe_LB_ITS.qza \
    --output-path ./OTU-ITS
    
biom convert \
    -i ./OTU-ITS/feature-table.biom \
    -o ./OTU-ITS/feature-table-97_guadeloupe_LB_ITS.tsv --to-tsv

# ouvrir le .tsv avec LibreOffice Calc, sélectionner les abondances, Edition>Find and Replace,
# chercher ^[0-9] et remplacer par & (cocher "regular expressions")

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Exported ./OTU-16S/table-97_guadeloupe_LB_16S.qza as BIOMV210DirFmt to directory ./OTU-16S
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Traceback (most recent call last):
  File "/opt/miniconda3/envs/qiime2-amplicon-2023.9/bin/biom", line 11, in <module>
    sys.exit(cli())
  File "/opt/miniconda3/envs/qiime2-amplicon-2023.9/lib/python3.8/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/miniconda3/envs/qiime2-amplicon-2023.9/lib/python3.8/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/miniconda3/envs/qiime2-amplicon-2023.9/lib/python3.8/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/miniconda3/envs/qiime2-amplicon-2023.9/lib/python3.8/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/miniconda3

: 1

* feature-classifier BLAST
* de préférence sur un cluster / un calculateur

In [4]:
#########################
## UNITE
## import de la base en artefact .qza
## source activate qiime2-amplicon-2023.9
cd /Volumes/BOUGREDANE/references_metabarcoding/ITS2/sh_qiime_release_s_all_04.04.2024
#mkdir ./qiime

qiime tools import \
  --type FeatureData[Sequence] \
  --input-path ./sh_refs_qiime_ver10_97_s_all_04.04.2024.fasta \
  --output-path ./sh_refs_qiime_ver10_97_s_all_04.04.2024.qza
  
qiime tools import \
  --type FeatureData[Taxonomy] \
  --input-path ./sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.txt \
  --output-path ./sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.qza
  
  #  --input-format HeaderlessTSVTaxonomyFormat \

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported ./sh_refs_qiime_ver10_97_s_all_04.04.2024.fasta as DNASequencesDirectoryFormat to ./sh_refs_qiime_ver10_97_s_all_04.04.2024.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Imported ./sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.txt as TSVTaxonomyDirectoryFormat to ./sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 


: 1

In [11]:
####################################
### ITS2 : Unite
## source activate qiime2-amplicon-2023.9
## mkdir ./Unite10

cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-ITS

qiime feature-classifier classify-consensus-blast \
    --i-query ./rep-seqs-97_guadeloupe_LB_ITS.qza \
    --i-reference-reads /Users/tonyrobinet/sync/references_metabarcoding/ITS2/sh_refs_qiime_ver10_97_s_all_04.04.2024.qza \
    --i-reference-taxonomy /Users/tonyrobinet/sync/references_metabarcoding/ITS2/sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.qza \
    --o-classification ./Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza \
    --o-search-results ./Unite10/top_hits_guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza

# exporter en .biom
qiime tools export \
    --input-path ./Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza \
    --output-path ./Unite10


(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureData[Taxonomy] to: ./Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza
Saved FeatureData[BLAST6] to: ./Unite10/top_hits_guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Exported ./Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza as TSVTaxonomyDirectoryFormat to directory ./Unite10
(qiime2-amplicon-2023.9) 


: 1

In [21]:
conda deactivate
source activate qiime2-shotgun-2023.9
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-ITS

qiime metadata tabulate \
--m-input-file rep-seqs-97_guadeloupe_LB_ITS.qza \
--m-input-file table_97_guadeloupe_LB_ITS.qza \
--m-input-file Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza \
--o-visualization merged.qzv

qiime tools export \
  --input-path merged.qzv \
  --output-path merged-data

(qiime2-shotgun-2023.9) 
(qiime2-shotgun-2023.9) 
(qiime2-shotgun-2023.9) 
(qiime2-shotgun-2023.9) 
There was an issue with merging QIIME 2 Metadata:

  Cannot merge because there are no IDs shared across metadata objects.

(qiime2-shotgun-2023.9) 
(qiime2-shotgun-2023.9) 
Usage: qiime tools export [OPTIONS]

  Exporting extracts (and optionally transforms) data stored inside an
  Artifact or Visualization. Note that Visualizations cannot be transformed
  with --output-format

Options:
  --input-path ARTIFACT/VISUALIZATION
                        Path to file that should be exported        [required]
  --output-path PATH    Path to file or directory where data should be
                        exported to                                 [required]
  --output-format TEXT  Format which the data should be exported as. This
                        option cannot be used with Visualizations
  --help                Show this message and exit.

                    There was a problem with the 

: 1

In [16]:
# BLAST+ 18S
# on aura préalablement entraîné le classifieur sur les amorces 18SV9 à partir de la base Silva 138.1
# voir le script classifier_rescript_18SV9.ipynb

cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-18S

qiime feature-classifier classify-sklearn \
   --i-reads ./rep-seqs-97_guadeloupe_LB_18S.qza \
   --i-classifier /Users/tonyrobinet/sync/references_metabarcoding/Silva138.1/silva-138.1-ssu-nr99-18SV9-classifier.qza \
   --p-n-jobs -1 \
   --output-dir ./OTU-18S-taxo

# exporter en .biom
qiime tools export \
    --input-path ./OTU-18S-taxo/classification.qza \
    --output-path ./OTU-18S-taxo

(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Saved FeatureData[Taxonomy] to: ./OTU-18S-taxo/classification.qza
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
(qiime2-amplicon-2023.9) 
Exported ./OTU-18S-taxo/classification.qza as TSVTaxonomyDirectoryFormat to directory ./OTU-18S-taxo
(qiime2-amplicon-2023.9) 


: 1

** On a ainsi créé, par marqueur :
- un tableau des OTUs à 3% (séquences et abondances des reads après filtres qualités) [feature-table_guyane_ITS2.tsv]
- un tableau des assignations taxonomiques des OTUs [taxonomy.tsv]